# Make training dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [56]:
train = pd.read_csv('../data/raw/ais_train.csv', sep='|')
#train =train.drop(['cog','sog','rot','heading','navstat','portId','etaRaw'], axis=1)
train =train.drop(['portId','etaRaw'], axis=1)
train['time'] = pd.to_datetime(train['time'])
train = train.sort_values(by=['vesselId','time'])
train.head()

# Load vessels data
vessels = pd.read_csv('../data/cleaned/cleaned_vessels.csv', sep=',')
vessels.head()

# Merge with training data
train = train.merge(vessels, on='vesselId', how='left')
train.head()
print(train.columns)


# Handle missing values in vessel data (if any)
# For demonstration, fill missing numerical values with median and categorical with mode
numerical_cols = ['CEU', 'DWT', 'GT', 'NT', 'breadth', 'depth', 'draft', 'enginePower', 'maxHeight', 'maxSpeed', 'maxWidth', 'rampCapacity', 'yearBuilt']
categorical_cols = ['vesselType', 'homePort', 'fuel', 'freshWater']

for col in numerical_cols:
    train[col] = train[col].fillna(train[col].median())

for col in categorical_cols:
    train[col] = train[col].fillna(train[col].mode()[0])

# Encode categorical variables
from sklearn.preprocessing import LabelEncoder

# Combine train and test vesselType categories to avoid unseen categories in test set
vessel_type_le = LabelEncoder()
train['vesselType_encoded'] = vessel_type_le.fit_transform(train['vesselType'])

home_port_le = LabelEncoder()
train['homePort_encoded'] = home_port_le.fit_transform(train['homePort'])

# Calculate vessel age at the time of each observation
train['vessel_age'] = train['time'].dt.year - train['yearBuilt']

# Drop original categorical columns if not needed
#train = train.drop(['vesselType', 'homePort', 'fuel', 'freshWater', 'yearBuilt'], axis=1)

# Reorder columns if desired
train.head()

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'latitude',
       'longitude', 'vesselId', 'shippingLineId', 'CEU', 'DWT', 'GT', 'NT',
       'vesselType', 'breadth', 'depth', 'draft', 'enginePower', 'freshWater',
       'fuel', 'homePort', 'length', 'maxHeight', 'maxSpeed', 'maxWidth',
       'rampCapacity', 'yearBuilt'],
      dtype='object')


,time,cog,sog,rot,heading,navstat,latitude,longitude,vesselId,shippingLineId,...,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt,vesselType_encoded,homePort_encoded,vessel_age
0,2024-01-12 14:07:47,308.1,17.1,-6,316,0,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61a8e672f9cba188601e84ab,...,OSLO,199.0,5.0,18.6,15.2,150.0,2000,2,29,24
1,2024-01-12 14:31:00,307.6,17.3,5,313,0,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61a8e672f9cba188601e84ab,...,OSLO,199.0,5.0,18.6,15.2,150.0,2000,2,29,24
2,2024-01-12 14:57:23,306.8,16.9,5,312,0,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61a8e672f9cba188601e84ab,...,OSLO,199.0,5.0,18.6,15.2,150.0,2000,2,29,24
3,2024-01-12 15:18:48,307.9,16.9,6,313,0,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61a8e672f9cba188601e84ab,...,OSLO,199.0,5.0,18.6,15.2,150.0,2000,2,29,24
4,2024-01-12 15:39:47,307.0,16.3,7,313,0,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61a8e672f9cba188601e84ab,...,OSLO,199.0,5.0,18.6,15.2,150.0,2000,2,29,24


In [57]:
print(train["vesselId"].value_counts())

# modify train so it only contains vesselId: 6323f2287abc89c0a9631e57 and 61e9f466b937134a3c4c0273

#train = train[train['vesselId'].isin(['6323f2287abc89c0a9631e57', '61e9f466b937134a3c4c0273'])]

6323f2287abc89c0a9631e57    8656
61e9f466b937134a3c4c0273    8626
61e9f464b937134a3c4c025f    7958
61e9f465b937134a3c4c026d    7928
61e9f465b937134a3c4c0271    7689
                            ... 
61e9f45cb937134a3c4c022b     196
61e9f42cb937134a3c4c00f9     191
61e9f3c6b937134a3c4bfed5     160
61e9f3adb937134a3c4bfe37      31
61e9f3cbb937134a3c4bff09       1
Name: vesselId, Length: 688, dtype: int64


In [52]:
def create_training_data(df: pd.DataFrame, N_KEEP_PAST: int) -> pd.DataFrame:
    data_rows = []  # List to collect all the data rows
    vessel_list = df['vesselId'].unique()
    for vessel in tqdm(vessel_list):
        vessel_data = df[df['vesselId'] == vessel].sort_values(by='time').reset_index(drop=True)
        num_rows = vessel_data.shape[0]
        if num_rows <= N_KEEP_PAST:
            continue  # Skip vessels with insufficient data
        for i in range(N_KEEP_PAST, num_rows):
            # Collect past N_KEEP_PAST locations and timestamps
            past_data = vessel_data.loc[i - N_KEEP_PAST:i - 1].reset_index(drop=True)
            current_data = vessel_data.loc[i]
            # Prepare a dictionary to hold the features and target
            data_row = {}
            target_time = current_data['time']
            for j in range(N_KEEP_PAST):
                past_time = past_data.loc[j, 'time']
                # Calculate the difference in minutes between past time and target time
                time_diff = (target_time - past_time).total_seconds() / 60.0  # Difference in minutes
                data_row[f'minutes_from_target_{j}'] = time_diff
                data_row[f'lat_{j}'] = past_data.loc[j, 'latitude']
                data_row[f'lon_{j}'] = past_data.loc[j, 'longitude']
                #######
                data_row[f'cog_{j}'] = past_data.loc[j, 'cog']
                data_row[f'sog_{j}'] = past_data.loc[j, 'sog']
                data_row[f'rot_{j}'] = past_data.loc[j, 'rot']
                data_row[f'heading_{j}'] = past_data.loc[j, 'heading']
                data_row[f'navstat_{j}'] = past_data.loc[j, 'navstat']
                ######
            for col in ['CEU', 'DWT', 'GT', 'NT', 'breadth', 'depth', 'draft',
                        'enginePower', 'maxHeight', 'maxSpeed', 'maxWidth', 'rampCapacity',
                        'vesselType_encoded', 'homePort_encoded', 'vessel_age']:
                data_row[col] = current_data[col]
            # Add current location as the target
            data_row['target_lat'] = current_data['latitude']
            data_row['target_lon'] = current_data['longitude']
            data_row['vesselId'] = vessel  # Include vesselId if needed
            data_row['target_time'] = target_time # Remove eventually
            # Append the row to the list
            data_rows.append(data_row)
    # Create final DataFrame from the list of data rows
    final_df = pd.DataFrame(data_rows)
    return final_df
        

In [60]:
N_KEEP_PAST = 5
final_train = create_training_data(train, N_KEEP_PAST)
final_train.head()

100%|██████████| 688/688 [18:08<00:00,  1.58s/it]


,minutes_from_target_0,lat_0,lon_0,cog_0,sog_0,rot_0,heading_0,navstat_0,minutes_from_target_1,lat_1,...,maxSpeed,maxWidth,rampCapacity,vesselType_encoded,homePort_encoded,vessel_age,target_lat,target_lon,vesselId,target_time
0,107.016667,7.50361,77.58340,308.1,17.1,-6,316,0,83.800000,7.57302,...,18.6,15.2,150.0,2,29,24,7.81285,77.18147,61e9f38eb937134a3c4bfd8b,2024-01-12 15:54:48
1,103.983333,7.57302,77.49505,307.6,17.3,5,313,0,77.600000,7.65043,...,18.6,15.2,150.0,2,29,24,7.86929,77.11032,61e9f38eb937134a3c4bfd8b,2024-01-12 16:14:59
2,98.016667,7.65043,77.39404,306.8,16.9,5,312,0,76.600000,7.71275,...,18.6,15.2,150.0,2,29,24,7.92585,77.03811,61e9f38eb937134a3c4bfd8b,2024-01-12 16:35:24
3,96.600000,7.71275,77.31394,307.9,16.9,6,313,0,75.616667,7.77191,...,18.6,15.2,150.0,2,29,24,7.98258,76.96880,61e9f38eb937134a3c4bfd8b,2024-01-12 16:55:24
4,94.816667,7.77191,77.23585,307.0,16.3,7,313,0,79.800000,7.81285,...,18.6,15.2,150.0,2,29,24,8.03598,76.90095,61e9f38eb937134a3c4bfd8b,2024-01-12 17:14:36


In [61]:
print(final_train.shape)
display(final_train.tail())

(1518629, 59)


,minutes_from_target_0,lat_0,lon_0,cog_0,sog_0,rot_0,heading_0,navstat_0,minutes_from_target_1,lat_1,...,maxSpeed,maxWidth,rampCapacity,vesselType_encoded,homePort_encoded,vessel_age,target_lat,target_lon,vesselId,target_time
1518624,97.083333,59.46124,22.12378,288.7,15.5,0,289,0,76.966667,59.48803,...,21.8,13.8,150.0,2,48,7,59.63337,21.43237,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 22:36:16
1518625,97.783333,59.48803,21.96346,288.5,15.4,7,291,0,77.200000,59.51857,...,21.8,13.8,150.0,2,48,7,59.69588,21.34225,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 22:57:05
1518626,98.016667,59.51857,21.80145,291.3,15.3,8,294,0,83.016667,59.54180,...,21.8,13.8,150.0,2,48,7,59.76388,21.35317,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 23:17:54
1518627,103.333333,59.54180,21.68877,292.6,14.6,3,295,0,82.933333,59.57721,...,21.8,13.8,150.0,2,48,7,59.83316,21.38489,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 23:38:13
1518628,103.733333,59.57721,21.54090,296.3,14.7,3,298,0,82.750000,59.63337,...,21.8,13.8,150.0,2,48,7,59.89167,21.54685,clh6aqawa0007gh0z9h6zi9bo,2024-05-07 23:59:01


# Modify Test dataset

In [62]:
test = pd.read_csv('../data/raw/ais_test.csv')
test['time'] = pd.to_datetime(test['time'])
test = test.merge(vessels, on='vesselId', how='left')
# Encode categorical variables in test data
test['vesselType_encoded'] = vessel_type_le.transform(test['vesselType'])
test['homePort_encoded'] = home_port_le.transform(test['homePort'])


for col in numerical_cols:
    test[col] = test[col].fillna(train[col].median())  # Use median from train

for col in categorical_cols:
    test[col] = test[col].fillna(train[col].mode()[0]) 
    
if 'yearBuilt' in test.columns:
    test['vessel_age'] = test['time'].dt.year - test['yearBuilt']
else:
    print("'yearBuilt' is not present in test data after merging.")
    # Decide how to handle this case
    # For example, you can set a default vessel age
    test['vessel_age'] = test['time'].dt.year - train['yearBuilt'].median()


In [63]:
print(test.shape)
# modify test so it only contains vesselId: 6323f2287abc89c0a9631e57 and 61e9f466b937134a3c4c0273
#test = test[test['vesselId'].isin(['6323f2287abc89c0a9631e57', '61e9f466b937134a3c4c0273'])]
print(test.shape)
display(test.head())

(51739, 24)
(51739, 24)


,ID,vesselId,time,scaling_factor,shippingLineId,CEU,DWT,GT,NT,vesselType,...,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt,vessel_age
0,0,61e9f3aeb937134a3c4bfe3d,2024-05-08 00:03:16,0.3,61a8e672f9cba188601e84ac,7934,31143.0,74255,18430.0,83.0,...,659.0,3668.0,VALLETTA,230.0,5.1,21.8,13.8,150.0,2011,13
1,1,61e9f473b937134a3c4c02df,2024-05-08 00:06:17,0.3,61be24574ea00ae59d0fe388,2500,13238.0,9984,18430.0,14.0,...,659.0,3668.0,Unknown,124.0,5.1,21.8,13.8,150.0,2012,12
2,2,61e9f469b937134a3c4c029b,2024-05-08 00:10:02,0.3,61ec6303a8cafc0e93f0e8f3,1400,7150.0,25995,18430.0,21.0,...,659.0,3668.0,PALERMO,186.0,5.1,21.8,13.8,150.0,2003,21
3,3,61e9f45bb937134a3c4c0221,2024-05-08 00:10:34,0.3,61be24564ea00ae59d0fe37a,5007,13951.0,45959,13788.0,83.0,...,346.0,2408.0,Monrovia,183.0,5.1,22.2,13.8,150.0,2011,13
4,4,61e9f38eb937134a3c4bfd8d,2024-05-08 00:12:27,0.3,61ec94f1a8cafc0e93f0e92a,4902,12325.0,46800,18430.0,83.0,...,659.0,3668.0,MONROVIA,182.0,5.1,21.8,13.8,150.0,2006,18


In [64]:
def create_test_features(train_df: pd.DataFrame, test_df: pd.DataFrame, N_KEEP_PAST: int) -> pd.DataFrame:
    data_rows = []  # List to collect all the data rows
    vessel_list = test_df['vesselId'].unique()
    
    for vessel in tqdm(vessel_list):
        # Get the test data for this vessel
        test_vessel_data = test_df[test_df['vesselId'] == vessel]
        for idx, test_row in test_vessel_data.iterrows():
            target_time = test_row['time']
            ID = test_row['ID']
            scaling_factor = test_row['scaling_factor']  # If needed later
            
            # Get past data from train for this vessel before the target time
            vessel_train_data = train_df[(train_df['vesselId'] == vessel) & (train_df['time'] < target_time)]
            vessel_train_data = vessel_train_data.sort_values(by='time').reset_index(drop=True)
            num_past_points = vessel_train_data.shape[0]
            
            if num_past_points < N_KEEP_PAST:
                # Not enough past data; decide how to handle (skip or pad with NaNs)
                continue  # Or handle as per your requirement
            
            # Get the last N_KEEP_PAST records
            past_data = vessel_train_data.iloc[-N_KEEP_PAST:].reset_index(drop=True)
            
            # Prepare a dictionary to hold the features
            data_row = {}
            for j in range(N_KEEP_PAST):
                past_time = past_data.loc[j, 'time']
                # Calculate the difference in minutes between past time and target time
                time_diff = (target_time - past_time).total_seconds() / 60.0  # Difference in minutes
                data_row[f'minutes_from_target_{j}'] = time_diff
                data_row[f'lat_{j}'] = past_data.loc[j, 'latitude']
                data_row[f'lon_{j}'] = past_data.loc[j, 'longitude']
                ###
                data_row[f'cog_{j}'] = past_data.loc[j, 'cog']
                data_row[f'sog_{j}'] = past_data.loc[j, 'sog']
                data_row[f'rot_{j}'] = past_data.loc[j, 'rot']
                data_row[f'heading_{j}'] = past_data.loc[j, 'heading']
                data_row[f'navstat_{j}'] = past_data.loc[j, 'navstat']
                ###
            for col in ['CEU', 'DWT', 'GT', 'NT', 'breadth', 'depth', 'draft',
                        'enginePower', 'maxHeight', 'maxSpeed', 'maxWidth', 'rampCapacity',
                        'vesselType_encoded', 'homePort_encoded', 'vessel_age']:
                data_row[col] = test_row[col]
            # Include vesselId and ID for result matching
            data_row['vesselId'] = vessel
            data_row['ID'] = ID
            # Append the row to the list
            data_rows.append(data_row)
    
    # Create test features DataFrame from the list of data rows
    test_features = pd.DataFrame(data_rows)
    return test_features

In [46]:
test_final = create_test_features(train, test, N_KEEP_PAST)

  0%|          | 0/215 [00:00<?, ?it/s]


KeyError: 'vesselType_encoded'

In [15]:
display(test_final.head())

,minutes_from_target_0,lat_0,lon_0,cog_0,sog_0,rot_0,heading_0,navstat_0,minutes_from_target_1,lat_1,...,minutes_from_target_4,lat_4,lon_4,cog_4,sog_4,rot_4,heading_4,navstat_4,vesselId,ID
0,789.100000,31.14645,-81.49792,349.9,0.0,0,344,5,783.066667,31.14645,...,15.000000,31.14647,-81.49789,179.6,0.0,0,344,5,61e9f3aeb937134a3c4bfe3d,0
1,822.066667,31.14645,-81.49792,349.9,0.0,0,344,5,816.033333,31.14645,...,47.966667,31.14647,-81.49789,179.6,0.0,0,344,5,61e9f3aeb937134a3c4bfe3d,143
2,837.033333,31.14645,-81.49792,349.9,0.0,0,344,5,831.000000,31.14645,...,62.933333,31.14647,-81.49789,179.6,0.0,0,344,5,61e9f3aeb937134a3c4bfe3d,282
3,858.066667,31.14645,-81.49792,349.9,0.0,0,344,5,852.033333,31.14645,...,83.966667,31.14647,-81.49789,179.6,0.0,0,344,5,61e9f3aeb937134a3c4bfe3d,426
4,876.066667,31.14645,-81.49792,349.9,0.0,0,344,5,870.033333,31.14645,...,101.966667,31.14647,-81.49789,179.6,0.0,0,344,5,61e9f3aeb937134a3c4bfe3d,551


In [16]:
final_train.to_csv('final_train.csv', index=False)
test_final.to_csv('test_final.csv', index=False)

# Machine learning part

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import lightgbm as lgb
from scipy.optimize import minimize

# Load the data
test_final = pd.read_csv('test_final.csv')
final_train = pd.read_csv('final_train.csv')



# Define features and targets
features = final_train.drop(columns=['target_lat', 'target_lon', 'vesselId', 'target_time'])
targets = final_train[['target_lat', 'target_lon']]

feature_columns = features.columns
X_test = test_final[feature_columns]

# Define the models
models = {
    'RandomForest': MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42, max_depth=4, n_jobs=8)),
    'XGBoost': MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, seed=42)),
    'ElasticNet': MultiOutputRegressor(ElasticNet(random_state=42)),
    'LightGBM': MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1))
}

# Prepare arrays to hold OOF predictions and test predictions
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

oof_preds = {model_name: np.zeros((features.shape[0], targets.shape[1])) for model_name in models}
test_preds = {model_name: np.zeros((X_test.shape[0], targets.shape[1], n_splits)) for model_name in models}

# Perform cross-validation and collect predictions
for fold, (train_idx, valid_idx) in enumerate(kf.split(features, targets)):
    X_train, y_train = features.iloc[train_idx], targets.iloc[train_idx]
    X_valid, y_valid = features.iloc[valid_idx], targets.iloc[valid_idx]
    
    for model_name, model in models.items():
        print(f"Training and predicting with model: {model_name}")
        clf = model
        clf.fit(X_train, y_train)
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        # Save OOF predictions
        oof_preds[model_name][valid_idx] = y_pred_valid
        # Save test predictions
        test_preds[model_name][:,:,fold] = y_pred_test

# Define the loss function for optimization
def mse_loss(weights):
    weights = np.array(weights)
    # Normalize weights to sum to 1
    weights = weights / np.sum(weights)
    # Combine OOF predictions using the weights
    final_oof = np.zeros_like(targets.values)
    for i, model_name in enumerate(models):
        final_oof += weights[i] * oof_preds[model_name]
    # Compute mean squared error
    mse = mean_squared_error(targets.values, final_oof)
    return mse

# Optimization methods to try
#methods = [
#    'Nelder-Mead', 'Powell', 'trust-constr', 'CG', 'BFGS', 'Newton-CG',
#    'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'dogleg', 'trust-ncg',
#    'trust-exact', 'trust-krylov'
#]
methods = [
    'Nelder-Mead', 'Powell',  'CG', 'BFGS',
    'L-BFGS-B', 'TNC', 'SLSQP', 
] # 'trust-constr' is just really slow and not performing well at the this moment

# Initial weights
initial_weights = np.ones(len(models)) / len(models)

# Constraints and bounds
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
bounds = [(0, 1)] * len(models)

# Optimize weights using different methods
best_mse = np.inf
best_weights = None
best_method = None

for method in methods:
    print(f"Optimizing weights using method: {method}")
    try:
        if method in ['trust-constr', 'COBYLA', 'SLSQP', 'trust-ncg', 'trust-krylov', 'trust-exact']:
            res = minimize(mse_loss, initial_weights, method=method, bounds=bounds, constraints=constraints)
        elif method in ['L-BFGS-B', 'TNC']:
            res = minimize(mse_loss, initial_weights, method=method, bounds=bounds)
        else:
            # For unconstrained methods, weights will be normalized in mse_loss
            res = minimize(mse_loss, initial_weights, method=method)
        if res.fun < best_mse:
            best_mse = res.fun
            best_weights = res.x / np.sum(res.x)  # Normalize weights
            best_method = method
        print(f"Method: {method}, MSE: {res.fun}")
    except Exception as e:
        print(f"Method: {method}, failed with error: {e}")

# Average test predictions over folds for each model
for model_name in models:
    test_preds[model_name] = np.mean(test_preds[model_name], axis=2)  # Average over folds

# Combine the test predictions using the best weights
final_test_pred = np.zeros((X_test.shape[0], targets.shape[1]))
for i, model_name in enumerate(models):
    final_test_pred += best_weights[i] * test_preds[model_name]

# Create a DataFrame with IDs and predictions
prediction_df = pd.DataFrame({
    'ID': test_final['ID'],
    'longitude_predicted': final_test_pred[:, 1],
    'latitude_predicted': final_test_pred[:, 0]
})

# Save the predictions to a CSV file
prediction_df.to_csv('predictions_ensemble2.csv', index=False)

# Print the best method and weights
print(f"Best optimization method: {best_method}")
print("Best weights:")
for i, model_name in enumerate(models):
    print(f"{model_name}: {best_weights[i]:.4f}")


Training and predicting with model: RandomForest
Training and predicting with model: XGBoost
Training and predicting with model: ElasticNet
Training and predicting with model: LightGBM
Training and predicting with model: RandomForest
Training and predicting with model: XGBoost
Training and predicting with model: ElasticNet
Training and predicting with model: LightGBM
Training and predicting with model: RandomForest
Training and predicting with model: XGBoost
Training and predicting with model: ElasticNet
Training and predicting with model: LightGBM
Training and predicting with model: RandomForest
Training and predicting with model: XGBoost
Training and predicting with model: ElasticNet
Training and predicting with model: LightGBM
Training and predicting with model: RandomForest
Training and predicting with model: XGBoost
Training and predicting with model: ElasticNet
Training and predicting with model: LightGBM
Optimizing weights using method: Nelder-Mead
Method: Nelder-Mead, MSE: 2.41

2.4149950074734456
